In [21]:
import csv
import os
import math
from tqdm import tqdm
import random
import re


random.seed(1234)

database_folder = "/scratch/acf15294oh/database"
test = "TM01_LSaug" #好きなラベルをつけてね

#data_csvフォルダがなかったら作成してください
train_file = f"./../data_csv/train_{test}.csv"  # 出力するCSVファイルのパス
val_file = f"./../data_csv/val_{test}.csv"
test_file = f"./../data_csv/test_{test}.csv"

#実験用データフォルダのパス+(old_text/wav...)になるように調整してください
old_eat_txt_folder = database_folder + f"/output_path/old_text_aug"  # txtフォルダのパス
old_eat_wav_folder = database_folder + f"/output_path/old_wav_aug"  # wavフォルダのパス"  



#0_eat_behavior...　で選択したマイクを指定。Augumentationでenhancementを選択した際には、conbinedを指定。英語の綴りが間違っていることは許して泣、
mics = [
#    "SM",
    "TM01",
#    "TM02",
#    "EM01",
#    "EM02",
#    "CM"
#     "conbined"
]

test_speaker = [
    "MAN01",
    "MDK01",
    "MDN01",
    "MKG01",
    "MHF01"
]

data = []
test_data =[] 

In [22]:
old_eat_wav_files = []
for root, dirs, files in os.walk(old_eat_wav_folder):
    for file_name in files:
        if file_name.endswith(".wav") and not file_name.startswith("._"):
            old_eat_wav_files.append(os.path.join(root, file_name))
old_eat_txt_files = []
for root, dirs, files in os.walk(old_eat_txt_folder):
    for file_name in files:
        if file_name.endswith(".txt") and not file_name.startswith("._"):
            old_eat_txt_files.append(os.path.join(root, file_name))

In [23]:
old_eat_txt_files[:3]

['/scratch/acf15294oh/database/behavior_4ch_conbined/old_text_aug/eat_MTK01_CBG_0048_87.txt',
 '/scratch/acf15294oh/database/behavior_4ch_conbined/old_text_aug/eat_MYH01_CBG_0325_48.txt',
 '/scratch/acf15294oh/database/behavior_4ch_conbined/old_text_aug/eat_MRT01_CBG_0099_29.txt']

In [24]:
old_eat_wav_files[:3]

['/scratch/acf15294oh/database/behavior_4ch_conbined/old_wav_aug/conbined/eat_MKS01_conbined_RTZ_2_0111_52.wav',
 '/scratch/acf15294oh/database/behavior_4ch_conbined/old_wav_aug/conbined/eat_MTG01_conbined_GUM_0142_84.wav',
 '/scratch/acf15294oh/database/behavior_4ch_conbined/old_wav_aug/conbined/eat_MTN02_conbined_RTZ_2_0202_88.wav']

In [25]:
speakers = []
test_speaker_check = []

for txt_file in old_eat_txt_files:
    test = False
    
    file = txt_file.split("/")[-1]  # ファイルパスからファイル名を取得

    speakers.append(file.split("_")[1])
    if file.split("_")[1] in test_speaker:
        test_speaker_check.append(file.split("_")[1])
        test = True

    with open(txt_file, "r") as txt:
        output = ""
        txt = txt.read().strip()
        for line in txt.split('\n'):
            label = line.split('\t')[2]
            if label == "ch":
                output += "#"
            elif label == "sw":
                output += "$"

    
    for mic in mics:
        parts = file.split("_")  # ファイル名を "_" で分割
        old_eat_file = parts[0] + "_" + parts[1] + "_" +mic + "_" + "_".join(parts[2:])  # 新しいファイル名を生成
        old_eat_file = old_eat_file.replace(".txt",".wav")
        file_id = old_eat_file
        old_eat_file = old_eat_wav_folder +"/"+ mic +"/" +old_eat_file
        
    
        ebd_data = output.replace(',', '').replace('、', '').replace('。', '').strip()
        ebd_data = " ".join(list(ebd_data))
        #wrd_data = " "*len(ebd_data)  
        wrd_data = ebd_data
            

        
        if os.path.exists(old_eat_file):
            if test:
                test_data.append([file_id, old_eat_file, wrd_data])
            else:
                data.append([file_id, old_eat_file, wrd_data])
        else:
            print(f"error: 存在しないのでスキップ {old_eat_file}")
            
set(test_speaker_check)

{'MAN01', 'MDK01', 'MDN01', 'MHF01', 'MKG01'}

In [26]:
print(len(data),len(test_data))

1950 400


In [27]:
len(set(speakers))

32

In [28]:
data

[['eat_MTK01_conbined_CBG_0048_87.wav',
  '/scratch/acf15294oh/database/behavior_4ch_conbined/old_wav_aug/conbined/eat_MTK01_conbined_CBG_0048_87.wav',
  '$ # # #'],
 ['eat_MYH01_conbined_CBG_0325_48.wav',
  '/scratch/acf15294oh/database/behavior_4ch_conbined/old_wav_aug/conbined/eat_MYH01_conbined_CBG_0325_48.wav',
  '# # # # # # # # # # $'],
 ['eat_MRT01_conbined_CBG_0099_29.wav',
  '/scratch/acf15294oh/database/behavior_4ch_conbined/old_wav_aug/conbined/eat_MRT01_conbined_CBG_0099_29.wav',
  '# # # # # # $ # # $'],
 ['eat_MRM01_conbined_RTZ_2_0042_28.wav',
  '/scratch/acf15294oh/database/behavior_4ch_conbined/old_wav_aug/conbined/eat_MRM01_conbined_RTZ_2_0042_28.wav',
  '# # # $ #'],
 ['eat_MYY01_conbined_CBG_0311_87.wav',
  '/scratch/acf15294oh/database/behavior_4ch_conbined/old_wav_aug/conbined/eat_MYY01_conbined_CBG_0311_87.wav',
  '# # # # # # # # # # # # # # # # # # # #'],
 ['eat_MYY01_conbined_GUM_0047_06.wav',
  '/scratch/acf15294oh/database/behavior_4ch_conbined/old_wav_aug/

In [29]:
datasets = data
len(datasets)

1950

In [30]:
datasets = random.sample(datasets, len(datasets))
valid_line = math.floor(len(datasets)*0.9)
train_data = datasets[:valid_line]
train_data.insert(0, ["ID", "wav", "wrd"])
valid_data = datasets[valid_line:]
valid_data.insert(0, ["ID", "wav", "wrd"])
test_data = test_data
test_data.insert(0, ["ID", "wav", "wrd"])


# CSVファイルにデータを書き込む
with open(train_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerows(train_data)



with open(val_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerows(valid_data)


with open(test_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerows(test_data)
    
    
print(f"CSVファイルを作成しました: {train_file}")
print(f"CSVファイルを作成しました: {val_file}")
print(f"CSVファイルを作成しました: {test_file}")




CSVファイルを作成しました: ./../data_csv/train_4ch_conbined.csv
CSVファイルを作成しました: ./../data_csv/val_4ch_conbined.csv
CSVファイルを作成しました: ./../data_csv/test_4ch_conbined.csv
